<a href="http://esprit.tn"><img src = "https://upload.wikimedia.org/wikipedia/commons/f/ff/Logo_ESPRIT_Ariana.jpg" width = 300, align = "center"></a>



<h1 align=center><font size = 5>Lab 9: Supervised learning -  Artificial Neural Networks (ANN) </font></h1>

Estimated duration: 50 min

## Goal

We aim through this lab to build a robust intrusion detection system (IDS) based on Artificial Neural Networks 

## Part I: Data loading and preparation 

In [1]:
import numpy as np
import pandas as pnd
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import Normalizer

from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error, roc_curve, classification_report,auc)


In [2]:
import datetime
t=datetime.datetime.now()
print(datetime.datetime.now()- t)

def init_chrono():
    return datetime.datetime.now()

def elapsed_time(t):
    if t:
        delta = init_chrono() - t
        print ('Elapsed time', delta)
        return delta 
    
# test
t = init_chrono()
elapsed_time(t)

0:00:00.000041
Elapsed time 0:00:00.000028


datetime.timedelta(microseconds=28)

In [3]:
print(sk.__version__)

0.20.1


In [4]:
data_path = './'
traindata = pnd.read_csv(data_path+'train.csv', header=None)
testdata = pnd.read_csv(data_path+'test.csv', header=None)

print('train_data', traindata.shape, '\n', 'test_data:', testdata.shape)

train_data (494021, 42) 
 test_data: (311029, 42)


In [5]:
testdata.head()

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,0,udp,private,SF,105,146,0,0,0,0,...,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0,normal.
1,0,udp,private,SF,105,146,0,0,0,0,...,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0,normal.
2,0,udp,private,SF,105,146,0,0,0,0,...,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0,normal.
3,0,udp,private,SF,105,146,0,0,0,0,...,254,1.0,0.01,0.00,0.0,0.0,0.0,0.0,0.0,snmpgetattack.
4,0,udp,private,SF,105,146,0,0,0,0,...,254,1.0,0.01,0.01,0.0,0.0,0.0,0.0,0.0,snmpgetattack.


#### Transforming categorical variables 

Class column transformation to binary

In [6]:
bin_func= lambda x : 1 if x!='normal.' else 0
bin_class= lambda x: 'normal' if x=='normal.' else 'attack' 

In [7]:
traindata[41]=traindata[41].map(bin_func)
testdata[41]=testdata[41].map(bin_func)

Showing the distribution of a given column

In [8]:
def show_dist(df_column, tag = 'Column'):
    if not df_column.empty: 
        dist = df_column.value_counts()
        print("%s distribution:\n" %tag, dist)

#Test
show_dist(traindata[41], tag="train dataset before ")  

train dataset before  distribution:
 1    396743
0     97278
Name: 41, dtype: int64


In [9]:
show_dist(traindata[41], 'Train_dataset:')
show_dist(testdata[41], 'Test_dataset:')

Train_dataset: distribution:
 1    396743
0     97278
Name: 41, dtype: int64
Test_dataset: distribution:
 1    250436
0     60593
Name: 41, dtype: int64


Retrieving categorical data  and transforming them to numerical representation.

In [10]:
def get_categorical(dataframe):
    types = dataframe.dtypes
    i = 0
    lst_cat=[]
    for var in types:
        if var == 'object':
            print(i)
            lst_cat.append(i)
        i=i+1
    return lst_cat

def handle_categorical(dataframe):
    if type(dataframe) != pnd.DataFrame:
        return None 

    lst= get_categorical(dataframe)
    
    print('%d variables are categorical %s ' %(len(lst), lst))
    for var_cat in lst:
        try:
            enc = sk.preprocessing.label.LabelEncoder()
            show_dist(dataframe[var_cat], 'Var[' + str(var_cat) + '] (before)')
            dataframe[var_cat] = enc.fit_transform(dataframe[var_cat])
            show_dist(dataframe[var_cat], 'Var[' + str(var_cat) + '] (after)')
        except Exception as e:
            print('Exception: Unable to convert categorical variable ', var_cat, e)
            
    #return dataframe
#handle_categorical(T)

In [11]:
handle_categorical(testdata)
handle_categorical(traindata)

1
2
3
3 variables are categorical [1, 2, 3] 
Var[1] (before) distribution:
 icmp    164969
tcp     119357
udp      26703
Name: 1, dtype: int64
Var[1] (after) distribution:
 0    164969
1    119357
2     26703
Name: 1, dtype: int64
Var[2] (before) distribution:
 ecr_i          164352
private         78510
http            41237
smtp             8268
pop_3            3972
domain_u         3160
ftp_data         2223
other            2185
telnet           2077
ftp               837
eco_i             547
imap4             333
finger            291
sunrpc            195
auth              150
time               82
ntp_u              80
echo               69
domain             68
bgp                66
link               66
remote_job         65
gopher             65
netstat            65
netbios_ns         64
ctf                63
urp_i              63
uucp               62
iso_tsap           62
nntp               62
                ...  
name               60
http_443           60
ldap        

In [12]:
print(traindata[41].value_counts())
print(testdata[41].value_counts())

1    396743
0     97278
Name: 41, dtype: int64
1    250436
0     60593
Name: 41, dtype: int64


#### Data extraction

In [13]:
## test 
X = traindata.iloc[:,0:41]
Y = traindata.iloc[:,41]
C = testdata.iloc[:,41]
T = testdata.iloc[:,0:41]

In [14]:

print(C.shape)

(311029,)


#### ETL: Data normalization

In [15]:
scaler = Normalizer().fit(X)
trainX = scaler.transform(X)

scaler = Normalizer().fit(T)
testT = scaler.transform(T)


traindata = np.array(trainX)
trainlabel = np.array(Y)

testdata = np.array(testT)
testlabel = np.array(C)


In [16]:
print(trainlabel[:15])
print (C.unique())

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 1]


## Part II: Desining the Neural network classification model 

In [17]:
import sklearn.neural_network as nn

In [18]:
t=init_chrono()
model = nn.MLPClassifier(activation='logistic' ,hidden_layer_sizes=(2,), max_iter=10,solver='sgd', verbose=True)

### Training the Neural network 

In [19]:
t=init_chrono()
model.fit(traindata, trainlabel)
print(model)
elapsed_time(t)

Iteration 1, loss = 0.48332956
Iteration 2, loss = 0.41307374
Iteration 3, loss = 0.32299056
Iteration 4, loss = 0.23579234
Iteration 5, loss = 0.17394126
Iteration 6, loss = 0.13444448
Iteration 7, loss = 0.10922281
Iteration 8, loss = 0.09261270
Iteration 9, loss = 0.08127847
Iteration 10, loss = 0.07328478
MLPClassifier(activation='logistic', alpha=0.0001, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(2,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=10, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='sgd', tol=0.0001,
       validation_fraction=0.1, verbose=True, warm_start=False)
Elapsed time 0:00:08.535900


/home/aminos/esprit/4nids/decision/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


datetime.timedelta(seconds=8, microseconds=535900)

### Using the trained model to make prediction 

In [20]:
# make predictions
expected = testlabel
predicted = model.predict(testdata)
np.savetxt('./predictedDT.txt', predicted, fmt='%01d')

In [24]:
print(metrics.confusion_matrix.__doc__)

Compute confusion matrix to evaluate the accuracy of a classification

    By definition a confusion matrix :math:`C` is such that :math:`C_{i, j}`
    is equal to the number of observations known to be in group :math:`i` but
    predicted to be in group :math:`j`.

    Thus in binary classification, the count of true negatives is
    :math:`C_{0,0}`, false negatives is :math:`C_{1,0}`, true positives is
    :math:`C_{1,1}` and false positives is :math:`C_{0,1}`.

    Read more in the :ref:`User Guide <confusion_matrix>`.

    Parameters
    ----------
    y_true : array, shape = [n_samples]
        Ground truth (correct) target values.

    y_pred : array, shape = [n_samples]
        Estimated targets as returned by a classifier.

    labels : array, shape = [n_classes], optional
        List of labels to index the matrix. This may be used to reorder
        or select a subset of labels.
        If none is given, those that appear at least once
        in ``y_true`` or ``y_pred`` are 

### Model evaluation

In [22]:


# summarize the fit of the model
avg= 'macro'
accuracy = accuracy_score(expected, predicted)
recall = recall_score(expected, predicted, average=avg)
precision = precision_score(expected, predicted , average=avg)
f1 = f1_score(expected, predicted , average=avg)

cm = metrics.confusion_matrix(expected, predicted)
print(cm)
tpr = float(cm[0][0])/np.sum(cm[0])
fpr = float(cm[1][1])/np.sum(cm[1])
print("%.3f" %tpr)
print("%.3f" %fpr)
print("Accuracy")
print("%.3f" %accuracy)
print("precision")
print("%.3f" %precision)
print("recall")
print("%.3f" %recall)
print("f-score")
print("%.3f" %f1)
print("fpr")
print("%.3f" %fpr)
print("tpr")
print("%.3f" %tpr)
print("***************************************************************")
elapsed_time(t)

[[ 57426   3167]
 [ 23506 226930]]
0.948
0.906
Accuracy
0.914
precision
0.848
recall
0.927
f-score
0.878
fpr
0.906
tpr
0.948
***************************************************************
Elapsed time 0:00:42.051979


datetime.timedelta(seconds=42, microseconds=51979)

 <hr>
Copyright &copy; Nov-2018 Dr.Amir Esseghir